<img src="../../img/python-logo-no-text.svg"
     style="display:block;margin:auto;width:10%"/>
<br>
<div style="text-align:center; font-size:200%;">
  <b>Dataclasses</b>
</div>
<br/>
<div style="text-align:center;">Dr. Matthias Hölzl</div>
<br/>
<div style="text-align:center;">module_200_object_orientation/topic_140_a3_dataclasses</div>

## Dataclasses

Definition of a class in which attributes are more visible, representation
and equality are predefined, etc.

The [documentation](https://docs.python.org/3/library/dataclasses.html)
includes other options.

In [1]:
from dataclasses import dataclass

In [2]:
@dataclass
class DataPoint:
    x: float
    y: float

In [3]:
dp = DataPoint(22, 33)
dp

DataPoint(x=22, y=33)

In [4]:
dp1 = DataPoint(1, 1)
dp2 = DataPoint(1, 1)

In [5]:
print(dp1 == dp2)
print(dp1 is dp2)

True
False


In [6]:
dp1.x = 2
dp1 == dp2

False

In [11]:
from dataclasses import field

In [12]:
@dataclass
class Point3D:
    x: float = field(default=0.0)
    y: float = field(default=0.0)
    z: float = field(default=0.0)
        
    def move(self, dx=0.0, dy=0.0, dz=0.0):
        self.x += dx
        self.y += dy
        self.z += dz

In [13]:
p3d = Point3D(1.0, 2.0)
p3d

Point3D(x=1.0, y=2.0, z=0.0)

In [14]:
p3d.move(dy=1.0, dz=3.0)
p3d

Point3D(x=1.0, y=3.0, z=3.0)

In [17]:
{(1, 2): "tuple", [1, 2]: "list"}

TypeError: unhashable type: 'list'

In [21]:
@dataclass(frozen=True)
class FrozenPoint:
    x: float = 0.0
    y: float = 0.0

In [22]:
fp = FrozenPoint()
fp

FrozenPoint(x=0.0, y=0.0)

In [23]:
{fp: "frozen"}

{FrozenPoint(x=0.0, y=0.0): 'frozen'}

In [25]:
fp.x = 1.0

FrozenInstanceError: cannot assign to field 'x'

In [27]:
from dataclasses import replace

In [28]:
fp

FrozenPoint(x=0.0, y=0.0)

In [30]:
replace(fp, x=1.0)

FrozenPoint(x=1.0, y=0.0)

In [31]:
replace(fp, x=1, y=2)

FrozenPoint(x=1, y=2)

Dataclasses ensure that default values are immutable (at least for some types...):

In [32]:
def add(x=0, y=0):
    return x + y

In [33]:
add()

0

In [34]:
add(1)

1

In [37]:
def my_append(value, lst=[]):
    lst.append(value)
    return lst

In [38]:
my_append(3, [])

[3]

In [39]:
my_append(3)

[3]

In [40]:
my_append(2)

[3, 2]

In [48]:
def better_append(value, lst=None):
    if lst is None:
        lst = list()
    lst.append(value)
    return lst

In [49]:
better_append(2)

[2]

In [50]:
better_append(3)

[3]

In [51]:
@dataclass
class DefaultDemo:
    items: list = field(default_factory=list)

In [52]:
d1 = DefaultDemo()
d2 = DefaultDemo()

In [77]:
d1.items.append(1)
print(d1)
print(d2)

DefaultDemo(items=[1, 1, 1])
DefaultDemo(items=[])


In [78]:
d1.my_attr = 1

In [80]:
d1.__dict__

{'items': [1, 1, 1], 'my_attr': 1}

However, the test for immutable defaults only works for some types from the standard library, not for user-defined types:

In [58]:
@dataclass
class BadDefault:
    point: Point3D = field(default_factory=Point3D)

In [59]:
bd1 = BadDefault()
bd2 = BadDefault()
bd1, bd2

(BadDefault(point=Point3D(x=0.0, y=0.0, z=0.0)),
 BadDefault(point=Point3D(x=0.0, y=0.0, z=0.0)))

In [60]:
bd1.point.move(1, 2)
bd1, bd2

(BadDefault(point=Point3D(x=1.0, y=2.0, z=0.0)),
 BadDefault(point=Point3D(x=0.0, y=0.0, z=0.0)))


It is possible to perform more complex initializations:

- The `__post_init__()` method can contain code that is executed after the
  generated `__init__()` method has been executed.
- The type `InitVar[T]` declasres that a class attribute serves as an argument
  for the `__post_init__()` method and not as an attribute for the instance.
- The keyword argument `init=False` for `field()` causes the corresponding
  attribute to not be initialized in the generated `__init__()` method.

In [62]:
from dataclasses import dataclass, field, InitVar

In [71]:
@dataclass
class DependentInit:
    x: InitVar[float]
    y: InitVar[float]
    z: InitVar[float]
    point: Point3D = field(init=False)
        
    def __post_init__(self, x, y, z):
        self.point = Point3D(x, y, z)

In [72]:
bd1 = DependentInit(0, 0, 0)
bd2 = DependentInit(x=1, y=2, z=3)
bd1, bd2

(DependentInit(point=Point3D(x=0, y=0, z=0)),
 DependentInit(point=Point3D(x=1, y=2, z=3)))

In [73]:
bd1.point.move(3, 5)
bd1, bd2

(DependentInit(point=Point3D(x=3, y=5, z=0.0)),
 DependentInit(point=Point3D(x=1, y=2, z=3)))

In [74]:
bd1.__dict__

{'point': Point3D(x=3, y=5, z=0.0)}

In [75]:
bd1.x

AttributeError: 'DependentInit' object has no attribute 'x'

## Workshop

- Notebook `workshop_062_objects`
- Section "Shopping list"